In [16]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import string
from nltk.corpus import stopwords
plt.ion()

In [45]:
%time
def clean_data(reviews):
    reviews.text = reviews.text.apply(lambda x: x.lower())
    
    def remove_punctuation(s):
    #s = s.split()
    
        s = [ch for ch in s if ch not in string.punctuation]

        
        s = "".join(s)
        
        s = [c for c in s if c == " " or c.isalnum()]
        
        s = "".join(s)
        
        return s
    
    
    def remove_stopwords(s):
        stop_words = set(stopwords.words("english"))

        s = s.split(" ")

        s = [word for word in s if word not in stop_words]
        
        
        s = " ".join(s)

        
        
        
        
        return s
    
    
    reviews.text = reviews.text.apply(lambda x:remove_punctuation(x))
    reviews.text = reviews.text.apply(lambda x:remove_stopwords(x))
    
    return reviews

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [28]:
%%time
reviews = pd.read_json("/home/bh_parijat/data/yelp/review.json",lines=True)

CPU times: user 1min 27s, sys: 25.7 s, total: 1min 53s
Wall time: 1min 52s


In [41]:
%%time
reviews.head()

CPU times: user 428 µs, sys: 41 µs, total: 469 µs
Wall time: 409 µs


,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_length
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,204
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,1561
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03,615
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,407
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,3509


In [49]:
%%time
reviews_cleaned = reviews.copy()
reviews_cleaned = clean_data(reviews_cleaned)

CPU times: user 26min 23s, sys: 54.9 s, total: 27min 18s
Wall time: 27min 18s


In [89]:
filter_null_row_cond = reviews_cleaned.text == "null"
reviews_cleaned = reviews_cleaned[~filter_null_row_cond]

In [90]:
reviews['text_length'] = reviews.text.apply(lambda x:len(x))

In [91]:
reviews_cleaned.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date', 'text_length', 'label'],
      dtype='object')

In [92]:
reviews_cleaned.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,text_length,label
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1,6,1,0,total bill horrible service 8gs crooks actuall...,2013-05-07 04:34:36,129,0
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5,0,0,0,adore travis hard rocks new kelly cardenas sal...,2017-01-14 21:30:33,1064,1
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5,3,0,0,say office really together organized friendly ...,2016-11-09 20:09:03,369,1
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5,0,0,0,went lunch steak sandwich delicious caesar sal...,2018-01-09 20:56:38,279,1
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1,7,0,0,today second three sessions paid although firs...,2018-01-30 23:07:38,2021,0


In [93]:
reviews.shape,reviews_cleaned.shape

((6685900, 10), (6685899, 11))

In [94]:
reviews_cleaned['label']= reviews.stars.apply(lambda x: 1 if x>=3 else 0)

In [95]:
%%time
reviews_cleaned.to_csv("model/master_reviews.csv",index=False)

CPU times: user 1min 56s, sys: 3.23 s, total: 1min 59s
Wall time: 2min 3s


In [63]:
# %%time
# reviews_saved = pd.read_csv("model/master_reviews.csv")

CPU times: user 43.9 s, sys: 604 ms, total: 44.5 s
Wall time: 44.5 s


In [69]:
#reviews_saved.shape,reviews_cleaned.shape

((6685900, 11), (6685900, 11))

<class 'str'> total bill horrible service 8gs crooks actually nerve charge us 69 3 pills checked online pills 19 cents avoid hospital ers costs
<class 'str'> adore travis hard rocks new kelly cardenas salon  im always fan great blowout stranger chains offer service however travis taken flawless blowout whole new level  

traviss greets perfectly green swoosh otherwise perfectly styled black hair vegasworthy rockstar outfit  next comes relaxing incredible shampoo  get full head message could cure even worst migraine minutes  scented shampoo room  travis freakishly strong fingers good way use perfect amount pressure  superb  starts glorious blowout one two three people involved best roundbrush action hair ever seen  team stylists clearly gets along extremely well evident way talk help one another really genuine corporate requirement  much fun 

next travis started flat iron  way flipped wrist get volume around without overdoing making look like texas pagent girl admirable  also worth not

<class 'str'> enjoy luxuries like electricity running water keep looking wont find cant tell many days electric water shut hours time looking place make home susceptible robbery place weeks ago hired painters come around paint doors painters came morning work left door open let paint dry wasnt home came home around 530 pm find door apartment completely wide open thankfully nothing stolen vandalized goes show inept complex  managers running safe responsible business reported incident management never received kind response unattended apartment left wide open 8 hours management couldnt bothered even call email situation 

to top last weekend call 911 halfnaked meth addict trying break car needed place sleep 3 cops firetruck ambulance ended come deal woman 3rd time 5 months call 911 drug addicts parking lot complex 

living isnt horrific experience dangerous live 

because living complete nightmare moving month early worth living somewhere dangerous this

ps  5 star reviews place written 

<class 'str'> ok place long never want daily special 7pm slow service decent food
<class 'str'> delish  need say  nice mean try flemings  great happy hour
<class 'str'> horrible experience stay away hotel changed rooms immediately got upgraded mini suite basically regular room nothing special super 8 would better bet 
gross gross gross 

you could listen review learn hard like beware
<class 'str'> recommendation fellow yelpers hubby came friday night preparty wine wine list california heavy great us fab blend niner estates cab seven falls need come back 2nd fave zin earthquake music kicks great vibe chatter around tables lively miss favorite wine shop 4 doors old home place feelibg right coukd easily regular small plates food looked greatwill come back try
<class 'str'> fantastic service lisa definitely recommend breakfast skillets   cant wait come back
<class 'str'> love good dog yep ill say im fan jjs char heel go dog like coney dog theyve got good selection beers well fountain drink

<class 'str'> came looking hockey bar reviews said place go um wasnt case would asked could turn one gave first beer arrived pretty much ignored place dead lot tvs showing 2 college football games pretty standard otherwise different pts lodge  couldnt say food didnt stick around wont back
<class 'str'> im sadout favorite chinese restaurant valley closed  drove way goodyear eat dinner arrived found totts closed sign said closed relocating almost started crying dream totts look forward making trek across town terrific food

i sure hope open back maybe goodyear 
<class 'str'> normally dont high expectations walmarts place even failed meet  biggest complaint  many items open walmarts locked cases  saw 097 shave cream locked case  wait ridiculous amount time get 875 item cant see nosehair clippers big shop lifting item  walmart near mountains edge  feeling shoplifting isnt issue stores area  parking lot rather small fullsized walmart  otherwise place new clean semiwell organized  didnt buy 

<class 'str'> found jpg via yelp checking website knew needed give call  jim came spent time walk different systems set installation day  gives everything need free monthly price competitive  would definitely recommend company others  longterm adt customer
<class 'str'> downside bluebird kitchen isnt closer office reason ive able try breakfast foods lunch isnt served 11 ive impressed everything ive 

their breakfast sandwiches hearty heavy pastries light satisfying love flavor combinations orange pistachio blueberry almond coconut cashew classic mixed berry top prices comparablelower chains cafés like one 

everything made inhouse even go dayold halfprice pastries still fresher anything else find area plus counter staff friendly without onobnoxious annoying bluebird kitchen definitely type restaurant pittsburgh needs
<class 'str'> good going  year  great food cooked order  juns special favorite  complaint prices went recently nothings changed cant understand food price could go buffet 

<class 'str'> 20 minute wait turned 2 hours called back saw small note saying provider medical doctor physicians assistant one point asked room could go lie wouldnt fall told keep rooms open emergencies wait dizzy feel like passing isnt emergent enough sheesh 2hour wait mere 4 patients waiting room ridiculous hadnt felt yucky would left unless make changes recommend finding another urgent care
<class 'str'> amazing gym  travel country frequently use dropins gym best ever come across  opened new location impressive  equipment high quality kept great condition 
some highlights me
plenty squat rack olympic lift stations
turf sleds kettlebells
trx pullup stations
assortment standard cable machines
dumbbells 150lbs im mistaken 
high quality cardio machines including 5 stair masters
clean
we also tribe fit small group fitness classes days week ive noticed vast improvement performance losing body fat well worth money
however offer classes included membership including spin yoga time thats unu

<class 'str'> know ive coming years go place good breakfast burrito problem servicefor part ok whenever call almost always put hold toss long leave thereusually min 2 mins 6 7 believe  tried day put hold waited waited6 mins later still response arrived location asked server said 2 thats theyre told put people hold  

that raises obvious questionwhy 2 servers extremely busy sat morningfrustrating customers got bit attitude server good measurenice
<class 'str'> frequented restaurant ever since first opened recent visits highly regarded weekend brunch  signing wife vip club printed 2 1 entree coupon decided use dinner  started bowl fresh taco chips 3 salsas  us decided chile rellenos entree  roasted poblano peppers enormous accompanied 2 types rice earthy refried beans service throughout good
<class 'str'> stephanie manicure pedicure months  done thorough job every time especially pedicure  started go great reviews disappointed
<class 'str'> pretty patient guy take cheap flight allegiant 

<class 'str'> ive done activity two times  first birthday present wife  sone friends town fun  difficult really control movement wind tunnel  second try yesterday  little easier  likely normal learning curve  nice town daughters wife joined  bought 10 flights package first visit also bought 10 yesterday recommend give discount 
the instructors cool enjoy activity genuinely enjoy teaching others  theres lot neat science aerodynamics biomechanics 
its cheap nice movies cant beat family fun activity  show designated time 30 min time fira video tutorial time waiting watch groups front fun experience planning go back
<class 'str'> every location consistently great  service excellent  like servers help one another  kale salad corn bread croutons really good  dressing salad yummy  fried green tomato pork belly ap delicious  tomato fried crispy  chicken honey hot sauce  always great
<class 'str'> good dinner fine beet salad vy good cornish hen lovely dessert inexpensive worth problemnoise loud

AttributeError: 'float' object has no attribute 'split'

In [ ]:
_ = reviews.tokenized.apply(lambda x:build_context_center(x,sla.pairs_map))

In [8]:
reviews[reviews.text.isnull()].shape

(65640, 10)

In [19]:
reviews.shape

(6685900, 10)

In [23]:
reviews.isna().head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,label
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


(6685900, 10)